## Lab 7: Eel Distribution Modeling with XGBoost

**Reference Paper:** [Elith et al. (2008)](https://ucsb.box.com/s/6k7636wsbogdg3orarxrlowke0ounbic)

In this lab, you will model the distribution of the eel species *Anguilla australis* using **boosted classification trees (BCTs)**, a machine learning technique that improves predictive performance by combining multiple decision trees. Elith et al. (2008) offered an early implementation of BRTs in an ecological setting to understand how environmental variables influence eel distribution.

You will work with **two datasets**:
1. **Training Data** – Used to build and evaluate your XGBoost model.
2. **Evaluation Data** – Used to assess model performance on unseen data.

To achieve the following objectives:
- Train and fine-tune an **XGBoost** model for classification of species presence/absence data.
- Compare your model’s performance to the approach used by Elith et al.


**Wherever applicable in this lab, use a random state of 808.**

### Step 0: Load libraries and data


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform, randint

# Download the datasets
model_data = pd.read_csv("/courses/EDS232/Data/model.data.csv").drop(columns=['Site'])
eval_data = pd.read_csv("/courses/EDS232/Data/eval.data.csv")

### Step 1:Initial Data Preprocessing
Let's get started by preparing our data. `Angaus` will be our target variable(`y`), and all other variables will be our features (`X`). Then encode the categorical feature using `LabelEncoder()`. The final step will be a bit different this time.  We don't need to split off testing data for the final model evaluation; a separate set (`eval_data`) will be used as in Elith et al.  We do, however, need to split our data in order to do the early stopping process. When splitting your data into training and validation, use a test size of 0.2 and a random state of 808. 

In [2]:
# View data structure 
print(model_data.head())

print(f"\n Data types: {model_data.dtypes}")

   Angaus  SegSumT  SegTSeas  SegLowFlow  DSDist  DSMaxSlope  USAvgT  \
0       0     16.0     -0.10       1.036   50.20        0.57    0.09   
1       1     18.7      1.51       1.003  132.53        1.15    0.20   
2       0     18.3      0.37       1.001  107.44        0.57    0.49   
3       0     16.7     -3.80       1.000  166.82        1.72    0.90   
4       1     17.2      0.33       1.005    3.95        1.15   -1.20   

   USRainDays  USSlope  USNative  DSDam    Method  LocSed  
0       2.470      9.8      0.81      0  electric     4.8  
1       1.153      8.3      0.34      0  electric     2.0  
2       0.847      0.4      0.00      0       spo     1.0  
3       0.210      0.4      0.22      1  electric     4.0  
4       1.980     21.9      0.96      0  electric     4.7  

 Data types: Angaus          int64
SegSumT       float64
SegTSeas      float64
SegLowFlow    float64
DSDist        float64
DSMaxSlope    float64
USAvgT        float64
USRainDays    float64
USSlope       flo

In [3]:
# Encode categorical feature, Method, using LabelEncoder
label_encoder = LabelEncoder()
model_data['Method'] = label_encoder.fit_transform(model_data['Method'])

# Repeat encode for eval data 
eval_data['Method'] = label_encoder.fit_transform(eval_data['Method'])

# Define features 
X = model_data.drop('Angaus', axis =1)
y = model_data['Angaus']


# Split data for the early stopping process
X_train, X_val , y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 808)


### Step 2: Determine best number of trees using early stopping
As a guard against overfitting while maximizing performance, we use **early stopping**. We start with a large number of trees and allow XGBoost to determine the optimal number by stopping training when the validation error no longer improves.

The choice of hyperparameter starting values is important in this process. We begin with:
- `n_estimators=1000` to ensure the model has enough capacity to learn meaningful patterns.
- `learning_rate=0.1` as a reasonable default that balances learning speed and performance.
- `eval_metric="logloss"` as the metric of performance to optimize.
- `early_stopping_rounds=50` to halt training if no improvement is seen for 50 rounds, preventing unnecessary computations.
- `random_state = 808`

We then `fit()` our specified baseline model, passing in the training sets as usual and specifying validation sets values for the `eval_set` parameter.

Finally, get and print the best number of trees from the fitted baseline model.


In [4]:
# Define XGBoost model 
model = xgb.XGBClassifier(n_estimators=  1000,
                              learning_rate=  0.1, 
                              early_stopping_rounds = 50,
                              eval_metric=  "logloss",
                              random_state = 808,
                              n_jobs=-1
                         )
# Fit model to training model 
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose = 0)

print(f"Best number of trees: {model.best_iteration}")
best_trees = model.best_iteration

Best number of trees: 34


### Step 3: Tune Learning Rate

The (`learning_rate` hyperparameter controls how much each tree contributes to improving the model's performance. A *higher* learning rate allows the model to learn quickly but risks missing the optimal solution and overfitting, while a *lower* learning rate makes learning slower but can improve generalization.

To find the optimal value, we'll use **randomized search cross-validation** (`RandomizedSearchCV`) to test different learning rates in the 0.01 to 0.3 range. Instead of testing every possible value, this method samples a set number of candidates (`n_iter`) from a defined parameter distribution.  In this case, sampling 20 candidates from a uniform distribution between `0.01` and `0.31`. Check out the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) on `scipy.stats.uniform` to see how it differs from `random.uniform`. Be sure to use a random state of 808.

After using `RandomizedSearchCV`, fit your model. Print the best learning rate. 


In [6]:
# Re define model 
model2 = xgb.XGBClassifier(n_estimators = best_trees,
                           eval_metric="logloss",
                           early_stopping_rounds=50,
                           random_state = 808, 
                           n_jobs = -1)

# Define parameters 
params = {
    "learning_rate": uniform(0.01, 0.3)
}

# Run random search 
random_search = RandomizedSearchCV(model2, 
                                  params, 
                                  n_iter=20, 
                                  scoring="accuracy",
                                  cv=5, 
                                  verbose =0, 
                                  random_state=808)

# Run random search 
random_search.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose = 0)

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=50,
                                           enable_categorical=False,
                                           eval_metric='logloss', gamma=None,
                                           gpu_id=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None, max_bin=No...
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None

In [7]:
# Determing best learning rate 
print(f'The best learning rate: {random_search.best_params_}')
best_learning_rate = random_search.best_params_['learning_rate']

The best learning rate: {'learning_rate': 0.07485018543633809}


### Step 4: Tune Tree-Specific Parameters

Now that we've determined the best number of tree and learning rate, we need to tune the complexity of individual trees in our model. Initialize your model with the best number of trees and learning rate.Then, define a parameter dictionary that takes on the following values:  

- `max_depth`(Controls how deep each tree can grow.  Takes integer values): A random integer from 3 to 10 ( inclusive of 3 and 10)
- `min_child_weight`( Determines the minimum number of samples required in a leaf node. Takes integer values) : A random integer from 1 to 10 ( inclusive of 1 and 10)
- `gamma` (Defines the minimum loss reduction needed to make a further split in a tree. Can take on values from a continuous range):  A uniform distribution from 0.05 to 0.10 - once again remember to check the `scipy.stats.uniform()` documentation! 
- `random_state = 808`

To find the best combination, we again use `RandomizedSearchCV`, allowing us to efficiently sample hyperparameters and evaluate different configurations using cross-validation. After fitting the model, print the best parameters. 



In [8]:
# Re define model 
model3 = xgb.XGBClassifier(n_estimators = best_trees,
                           learning_rate = best_learning_rate,
                           eval_metric="logloss",
                           early_stopping_rounds=50,
                           random_state = 808, 
                           n_jobs = -1)

# Define parameters 
params = {
    "max_depth": randint(3,11),
    "min_child_weight": randint(1,11),  
    "gamma": uniform(0.05,0.1)
}

# Run random search 
random_search2 = RandomizedSearchCV(model3, 
                                  params, 
                                  n_iter=20, 
                                  scoring="accuracy",
                                  cv=5, 
                                  verbose =0, 
                                  random_state=808)
# Run random search 
random_search2.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose = 0)

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=50,
                                           enable_categorical=False,
                                           eval_metric='logloss', gamma=None,
                                           gpu_id=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=0.0748501854363...
                                           predictor=None, random_state=808,
                                           reg_alpha=None, reg

In [9]:
# View best parameters 
print(f'Best parameters: {random_search2.best_params_}')
best_tree_parameters = random_search2.best_params_

Best parameters: {'gamma': 0.09587121646965052, 'max_depth': 5, 'min_child_weight': 4}


### Step 5: Tune Stochastic Components

Now, we are finally ready to tune the stochastic components of the XGBoost model.  These parameters help prevent overfitting by reducing correlation between trees. Initialize your model with the best number of trees, best learning rate,and your optimized tree values (**Note**: you can use \**best_tree_parameters to unpack the the dictionary of optimzed tree values) .Then, define a parameter dictionary that takes on the following values:  

- `subsample` (Controls the fraction of training samples used for each boosting round) : A uniform distribution between .5 and .10 (remeber to check `scipy.stats.uniform()` documentation! )
- `colsample_bytree`(Specifies the fraction of features to consider when building each tree) : A uniform distribution between .5 and .10
- `random_state = 808`

We again use `RandomizedSearchCV` to find the best combination of these parameters. After fitting the model, print the best parameters. 


In [10]:
# Re define model 
model4 = xgb.XGBClassifier(n_estimators = best_trees,
                           learning_rate = best_learning_rate,
                           **best_tree_parameters,
                           early_stopping_rounds=50,
                           random_state = 808, 
                           n_jobs = -1)

# Define parameters 
params = {
    "subsample": uniform(0.5,0.5),
    "colsample_bytree": uniform(0.5,0.5)
}

# Run random search 
random_search3 = RandomizedSearchCV(model4, 
                                  params, 
                                  n_iter=20, 
                                  scoring="accuracy",
                                  cv=5, 
                                  verbose =0, 
                                  random_state=808)
# Run random search 
random_search3.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose = 0)

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=50,
                                           enable_categorical=False,
                                           eval_metric=None,
                                           gamma=0.09587121646965052,
                                           gpu_id=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=0.074...
                                           n_estimators=34, n_jobs=-1,
                         

In [11]:
# View best parameters 
print(f'Best parameters: {random_search3.best_params_}')
best_stoc_parameters = random_search3.best_params_

Best parameters: {'colsample_bytree': 0.7672505595364525, 'subsample': 0.617966164783058}


### Step 6: Final Model Training and Evaluation

With the best hyperparameters selected, we now train the final model on the full training dataset and evaluate it on the separate evaluation dataset.

1. Prepare the evaluation data in the same manner as you did the training data

2. Train final model using the best parameters found in previous tuning steps (`best_tree_params`, `best_stochastic_params`).Set  `eval_metric = "logloss"` 

3. Fit the model to the full training dataset and predict on the evaluation data 


In [12]:
# Define features 
X_eval = eval_data.drop('Angaus_obs', axis = 1)
y_eval = eval_data['Angaus_obs']

# Train model using best parameters 
model_best = xgb.XGBClassifier(n_estimators = best_trees,
                           learning_rate = best_learning_rate,
                           **best_tree_parameters,
                           **best_stoc_parameters, 
                           early_stopping_rounds=50,
                           random_state = 808, 
                           n_jobs = -1)
# Fit model to training data 
model_best.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose = 0)

# Predict on eval data 
preds = model_best.predict(X_eval)

### Step 7: Model Performance

Compute and print the AUC and feature importances for your model.

In [13]:
# Get probability predictions 
pred_probs = model_best.predict_proba(X_eval)[:, 1]

# Calculate ROC & AUC score
auc_score = roc_auc_score(y_eval, pred_probs)
print(f"AUC Score: {auc_score:.3f}")

# Determine feature importance
feature_importance = pd.DataFrame({'Feature': X_train.columns, 
                                   'Importance': model_best.feature_importances_})
# Sort by importance
feature_importance = feature_importance.sort_values(by="Importance", ascending=False)
feature_importance

AUC Score: 0.870


,Feature,Importance
0,SegSumT,0.209073
10,Method,0.119984
6,USRainDays,0.101490
8,USNative,0.100804
4,DSMaxSlope,0.083794
7,USSlope,0.072961
1,SegTSeas,0.069668
3,DSDist,0.069405
5,USAvgT,0.068352
11,LocSed,0.054479


### Step 8: The comparison
How does your model's performance compare to the of Elith et al. (See Tables 2 and 3)?  Is there another way to compare the models in addition to predictive performance?  Whose model wins in that regard?

In terms of feature importance, my model and the model of the paper get most of the same answers, with `SegSumT` being the most important feature and `DSDam` being the least important. However, the rest of the order is a bit different. In terms of model performance, my model has a AUC score of 0.874 and the paper evaluates a score of 0·869. While the AUC is one of the most common ways to compare models, there are other strategies such as comparing interpretability and robustness. 